In [1]:
import pandas as pd
import pyspark.sql.functions as F
from pyspark.sql import Window
import re
from pyspark.sql.types import *
import unicodedata
from google.cloud import storage
from functools import reduce
from pyspark.sql import DataFrame
import datetime
# spark.conf.set("spark.sql.crossJoin.enabled", True)

<hr />

### google mobility preprocessing

<hr />

In [2]:
# read data from gcs
gmr = spark.read.csv('gs://ai-covid19-datalake/raw/google-mobility/google-mobility_report-28-05-2021.csv', inferSchema=True, header=True)

In [3]:
# head? 

gmr.limit(5).toPandas()

,country_region_code,country_region,sub_region_1,sub_region_2,metro_area,iso_3166_2_code,census_fips_code,place_id,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
0,AE,United Arab Emirates,None,None,None,None,None,ChIJvRKrsd9IXj4RpwoIwFYv0zM,2020-02-15,0,4,5,0,2,1
1,AE,United Arab Emirates,None,None,None,None,None,ChIJvRKrsd9IXj4RpwoIwFYv0zM,2020-02-16,1,4,4,1,2,1
2,AE,United Arab Emirates,None,None,None,None,None,ChIJvRKrsd9IXj4RpwoIwFYv0zM,2020-02-17,-1,1,5,1,2,1
3,AE,United Arab Emirates,None,None,None,None,None,ChIJvRKrsd9IXj4RpwoIwFYv0zM,2020-02-18,-2,1,5,0,2,1
4,AE,United Arab Emirates,None,None,None,None,None,ChIJvRKrsd9IXj4RpwoIwFYv0zM,2020-02-19,-2,0,4,-1,2,1


In [4]:
# schema
gmr.printSchema()

root
 |-- country_region_code: string (nullable = true)
 |-- country_region: string (nullable = true)
 |-- sub_region_1: string (nullable = true)
 |-- sub_region_2: string (nullable = true)
 |-- metro_area: string (nullable = true)
 |-- iso_3166_2_code: string (nullable = true)
 |-- census_fips_code: integer (nullable = true)
 |-- place_id: string (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- retail_and_recreation_percent_change_from_baseline: integer (nullable = true)
 |-- grocery_and_pharmacy_percent_change_from_baseline: integer (nullable = true)
 |-- parks_percent_change_from_baseline: integer (nullable = true)
 |-- transit_stations_percent_change_from_baseline: integer (nullable = true)
 |-- workplaces_percent_change_from_baseline: integer (nullable = true)
 |-- residential_percent_change_from_baseline: integer (nullable = true)



In [5]:
# change timestamp var type to date type (we do not need hours or minutes)
gmr = gmr.withColumn('date', F.col('date').cast('date'))

In [6]:
gmr.printSchema()

root
 |-- country_region_code: string (nullable = true)
 |-- country_region: string (nullable = true)
 |-- sub_region_1: string (nullable = true)
 |-- sub_region_2: string (nullable = true)
 |-- metro_area: string (nullable = true)
 |-- iso_3166_2_code: string (nullable = true)
 |-- census_fips_code: integer (nullable = true)
 |-- place_id: string (nullable = true)
 |-- date: date (nullable = true)
 |-- retail_and_recreation_percent_change_from_baseline: integer (nullable = true)
 |-- grocery_and_pharmacy_percent_change_from_baseline: integer (nullable = true)
 |-- parks_percent_change_from_baseline: integer (nullable = true)
 |-- transit_stations_percent_change_from_baseline: integer (nullable = true)
 |-- workplaces_percent_change_from_baseline: integer (nullable = true)
 |-- residential_percent_change_from_baseline: integer (nullable = true)



#### Next steps: 
* select brazillian observations, keeping the header
* make a rapid descriptive analysis
* take out special characters
* upper case the text values
* take the sufix from states names
* write the preprocessed data in a proper location
* make another descriptive analysis

<hr />

In [7]:
# select brazillian observations, keeping the header
print(gmr.count())
gmr = gmr.filter(F.col('country_region_code')=='BR')
print(gmr.count())

5561418
913701


<hr />

#### make a rapid descriptive analysis 
* what are the states? 
* how much cities? 
* how much cities from each states? 
* how much observavions from each state? 
* max and min period for observations of each state
* null values per state on variables: census_fips_code, residential_percent_change_from_baseline, workplaces_percent_change_from_baseline, transit_stations_percent_change_from_baseline, parks_percent_change_from_baseline, and retail_and_recreation_percent_change_from_baseline

In [8]:
# what are the states?
# how much cities?
print("how much cities:", gmr.select(['sub_region_2']).distinct().count())

# # how much cities from each states?
states_and_cities = gmr.select(['sub_region_1', 'sub_region_2']).distinct().groupby('sub_region_1').count().sort('sub_region_1').withColumnRenamed('count', 'n_cities')
# states_and_cities.show(30)

# # how much observavions from each state?
n_observations = gmr.select(['sub_region_1', 'sub_region_2']).groupby('sub_region_1').count().sort('sub_region_1').withColumnRenamed('count', 'n_observations')
# n_observations.show(30)

# # max and min period for observations of each state
w = Window.partitionBy('sub_region_1')
min_date = gmr.select(['sub_region_1', 'date']).withColumn('min_date', F.min('date').over(w)).where(F.col('date') == F.col('min_date')).distinct().drop('date')
max_date = gmr.select(['sub_region_1', 'date']).withColumn('max_date', F.max('date').over(w)).where(F.col('date') == F.col('max_date')).distinct().drop('date')
# min_date.show(30)
# max_date.show(30)

# null values
columns = ['census_fips_code', 'residential_percent_change_from_baseline', 'workplaces_percent_change_from_baseline', 'transit_stations_percent_change_from_baseline', 'parks_percent_change_from_baseline', 'retail_and_recreation_percent_change_from_baseline']

nulls_census_fips_code = gmr.filter(F.col('census_fips_code').isNull()).select(['sub_region_1', 'census_fips_code']).groupBy(['sub_region_1']).count().withColumnRenamed('count', 'nulls_census_fips_code')
nulls_residential_percent_change_from_baseline = gmr.filter(F.col('residential_percent_change_from_baseline').isNull()).select(['sub_region_1', 'residential_percent_change_from_baseline']).groupBy(['sub_region_1']).count().withColumnRenamed('count', 'null_residential_percent_change_from_baseline')
nulls_workplaces_percent_change_from_baseline = gmr.filter(F.col('workplaces_percent_change_from_baseline').isNull()).select(['sub_region_1', 'workplaces_percent_change_from_baseline']).groupBy(['sub_region_1']).count().withColumnRenamed('count', 'workplaces_percent_change_from_baseline')
nulls_transit_stations_percent_change_from_baseline = gmr.filter(F.col('transit_stations_percent_change_from_baseline').isNull()).select(['sub_region_1', 'transit_stations_percent_change_from_baseline']).groupBy(['sub_region_1']).count().withColumnRenamed('count', 'transit_stations_percent_change_from_baseline')
nulls_parks_percent_change_from_baseline = gmr.filter(F.col('parks_percent_change_from_baseline').isNull()).select(['sub_region_1', 'parks_percent_change_from_baseline']).groupBy(['sub_region_1']).count().withColumnRenamed('count', 'parks_percent_change_from_baseline')
nulls_retail_and_recreation_percent_change_from_baseline = gmr.filter(F.col('retail_and_recreation_percent_change_from_baseline').isNull()).select(['sub_region_1', 'retail_and_recreation_percent_change_from_baseline']).groupBy(['sub_region_1']).count().withColumnRenamed('count', 'retail_and_recreation_percent_change_from_baseline')

nulls = nulls_census_fips_code.join(nulls_residential_percent_change_from_baseline, 'sub_region_1', how='left')\
                                .join(nulls_workplaces_percent_change_from_baseline, 'sub_region_1', how='left')\
                                .join(nulls_transit_stations_percent_change_from_baseline, 'sub_region_1', how='left')\
                                .join(nulls_parks_percent_change_from_baseline, 'sub_region_1', how='left')\
                                .join(nulls_retail_and_recreation_percent_change_from_baseline, 'sub_region_1', how='left')
                                

# joining all
desc_gmr = states_and_cities.join(n_observations, 'sub_region_1', 'left')\
                            .join(min_date, 'sub_region_1', 'left')\
                            .join(max_date, 'sub_region_1', 'left')\
                            .join(nulls, 'sub_region_1', 'left')

# writing and showing

desc_gmr.write.csv('gs://ai-covid19-datalake/raw/google-mobility/desc_pp_google_mobility_report_28_05_2021_pre_formatting_v1.csv', header=True, mode='overwrite')

desc_gmr.toPandas()

how much cities: 2254


,sub_region_1,n_cities,n_observations,min_date,max_date,nulls_census_fips_code,null_residential_percent_change_from_baseline,workplaces_percent_change_from_baseline,transit_stations_percent_change_from_baseline,parks_percent_change_from_baseline,retail_and_recreation_percent_change_from_baseline
0,State of Minas Gerais,311,118887.0,2020-02-15,2021-05-28,118887.0,72278.0,3956.0,86737.0,88895.0,80875.0
1,State of Espírito Santo,60,22671.0,2020-02-15,2021-05-28,22671.0,14158.0,622.0,16770.0,13865.0,13729.0
2,State of Mato Grosso,56,21752.0,2020-02-15,2021-05-28,21752.0,13063.0,384.0,15115.0,16370.0,14029.0
3,State of Goiás,92,37585.0,2020-02-15,2021-05-28,37585.0,21551.0,2231.0,25683.0,26847.0,21676.0
4,State of Rio de Janeiro,86,36857.0,2020-02-15,2021-05-28,36857.0,12921.0,895.0,17468.0,14593.0,14025.0
5,State of Roraima,3,1222.0,2020-02-15,2021-05-28,1222.0,284.0,NaN,334.0,334.0,284.0
6,State of Ceará,106,40584.0,2020-02-15,2021-05-28,40584.0,27520.0,1235.0,35906.0,26200.0,29591.0
7,State of Paraná,183,69688.0,2020-02-15,2021-05-28,69688.0,40760.0,3171.0,48378.0,51146.0,44475.0
8,State of Paraíba,50,18062.0,2020-02-15,2021-05-28,18062.0,12976.0,1317.0,14891.0,13000.0,13167.0
9,State of Pará,72,27842.0,2020-02-15,2021-05-28,27842.0,17859.0,1407.0,20428.0,16362.0,18925.0


<hr />

In [9]:
# take out special characters
def remove_accents(col):
    col = str(col)
    col = col.replace('á', 'a')
    col = col.replace('à', 'a')
    col = col.replace('ã', 'a')
    col = col.replace('â', 'a')
    col = col.replace('é', 'e')
    col = col.replace('ê', 'e')
    col = col.replace('í', 'i')
    col = col.replace('ó', 'o')
    col = col.replace('õ', 'o')
    col = col.replace('ô', 'o')
    col = col.replace('ú', 'u')
    col = col.replace('ü', 'u')
    col = col.replace('ç', 'c')
    return col
udf_remove_accents = F.udf(remove_accents, StringType())

def normalize_ascii(col):
    return re.sub('[^A-Za-z0-9]+', ' ', str(col))
udf_normalize_ascii = F.udf(normalize_ascii, StringType())

def take_state_prefix(col):
    return col.replace('STATE OF ', '')
udf_take_state_prefix = F.udf(take_state_prefix, StringType())

<hr />

In [10]:
# dropping NA in state and municipality
gmr = gmr.dropna(subset=['sub_region_1', 'sub_region_2'])

# taking accents and non-ascii chars
gmr = gmr.withColumn('sub_region_1', udf_remove_accents(F.col('sub_region_1')))
gmr = gmr.withColumn('sub_region_1', udf_normalize_ascii(F.col('sub_region_1')))

gmr = gmr.withColumn('sub_region_2', udf_remove_accents(F.col('sub_region_2')))
gmr = gmr.withColumn('sub_region_2', udf_normalize_ascii(F.col('sub_region_2')))


# turning columns into upper case
gmr = gmr.withColumn('sub_region_1', F.upper(F.col('sub_region_1')))
gmr = gmr.withColumn('sub_region_2', F.upper(F.col('sub_region_2')))

# taking prefix
gmr = gmr.withColumn('sub_region_1', udf_take_state_prefix(F.col('sub_region_1')))

In [11]:
gmr.select(['sub_region_1', 'sub_region_2']).distinct().show()

+-----------------+------------------+
|     sub_region_1|      sub_region_2|
+-----------------+------------------+
|          ALAGOAS|  SAO JOSE DA LAJE|
|            BAHIA|         IBIRATAIA|
|            CEARA|      QUIXERAMOBIM|
|RIO GRANDE DO SUL|             GIRUA|
|   SANTA CATARINA|DIONISIO CERQUEIRA|
|          PARAIBA|         BOQUEIRAO|
|       PERNAMBUCO|        CAMARAGIBE|
|        SAO PAULO|             AVARE|
|        SAO PAULO|            ITAPUI|
|        SAO PAULO|           ITUPEVA|
|   ESPIRITO SANTO|        JOAO NEIVA|
|      MATO GROSSO|            MATUPA|
|     MINAS GERAIS|       MATEUS LEME|
|            BAHIA|            IBICUI|
|            BAHIA|             LAPAO|
|            CEARA|            AURORA|
|            CEARA|         CARIRIACU|
|            CEARA|       NOVA RUSSAS|
|     MINAS GERAIS|      ITAMARANDIBA|
|     MINAS GERAIS|      JUIZ DE FORA|
+-----------------+------------------+
only showing top 20 rows



In [12]:
# what are the states?
# how much cities?
print("how much cities:", gmr.select(['sub_region_2']).distinct().count())

# # how much cities from each states?
states_and_cities = gmr.select(['sub_region_1', 'sub_region_2']).distinct().groupby('sub_region_1').count().sort('sub_region_1').withColumnRenamed('count', 'n_cities')
# states_and_cities.show(30)

# # how much observavions from each state?
n_observations = gmr.select(['sub_region_1', 'sub_region_2']).groupby('sub_region_1').count().sort('sub_region_1').withColumnRenamed('count', 'n_observations')
# n_observations.show(30)

# # max and min period for observations of each state
w = Window.partitionBy('sub_region_1')
min_date = gmr.select(['sub_region_1', 'date']).withColumn('min_date', F.min('date').over(w)).where(F.col('date') == F.col('min_date')).distinct().drop('date')
max_date = gmr.select(['sub_region_1', 'date']).withColumn('max_date', F.max('date').over(w)).where(F.col('date') == F.col('max_date')).distinct().drop('date')
# min_date.show(30)
# max_date.show(30)

# null values
columns = ['census_fips_code', 'residential_percent_change_from_baseline', 'workplaces_percent_change_from_baseline', 'transit_stations_percent_change_from_baseline', 'parks_percent_change_from_baseline', 'retail_and_recreation_percent_change_from_baseline']

nulls_census_fips_code = gmr.filter(F.col('census_fips_code').isNull()).select(['sub_region_1', 'census_fips_code']).groupBy(['sub_region_1']).count().withColumnRenamed('count', 'nulls_census_fips_code')
nulls_residential_percent_change_from_baseline = gmr.filter(F.col('residential_percent_change_from_baseline').isNull()).select(['sub_region_1', 'residential_percent_change_from_baseline']).groupBy(['sub_region_1']).count().withColumnRenamed('count', 'null_residential_percent_change_from_baseline')
nulls_workplaces_percent_change_from_baseline = gmr.filter(F.col('workplaces_percent_change_from_baseline').isNull()).select(['sub_region_1', 'workplaces_percent_change_from_baseline']).groupBy(['sub_region_1']).count().withColumnRenamed('count', 'workplaces_percent_change_from_baseline')
nulls_transit_stations_percent_change_from_baseline = gmr.filter(F.col('transit_stations_percent_change_from_baseline').isNull()).select(['sub_region_1', 'transit_stations_percent_change_from_baseline']).groupBy(['sub_region_1']).count().withColumnRenamed('count', 'transit_stations_percent_change_from_baseline')
nulls_parks_percent_change_from_baseline = gmr.filter(F.col('parks_percent_change_from_baseline').isNull()).select(['sub_region_1', 'parks_percent_change_from_baseline']).groupBy(['sub_region_1']).count().withColumnRenamed('count', 'parks_percent_change_from_baseline')
nulls_retail_and_recreation_percent_change_from_baseline = gmr.filter(F.col('retail_and_recreation_percent_change_from_baseline').isNull()).select(['sub_region_1', 'retail_and_recreation_percent_change_from_baseline']).groupBy(['sub_region_1']).count().withColumnRenamed('count', 'retail_and_recreation_percent_change_from_baseline')

nulls = nulls_census_fips_code.join(nulls_residential_percent_change_from_baseline, 'sub_region_1', how='left')\
                                .join(nulls_workplaces_percent_change_from_baseline, 'sub_region_1', how='left')\
                                .join(nulls_transit_stations_percent_change_from_baseline, 'sub_region_1', how='left')\
                                .join(nulls_parks_percent_change_from_baseline, 'sub_region_1', how='left')\
                                .join(nulls_retail_and_recreation_percent_change_from_baseline, 'sub_region_1', how='left')
                                

# joining all
desc_gmr = states_and_cities.join(n_observations, 'sub_region_1', 'left')\
                            .join(min_date, 'sub_region_1', 'left')\
                            .join(max_date, 'sub_region_1', 'left')\
                            .join(nulls, 'sub_region_1', 'left')

# writing and showing

desc_gmr.write.csv('gs://ai-covid19-datalake/standard/google-mobility/desc_pp_google_mobility_report_28_05_2021_pos_formatting_v1.csv', header=True, mode='overwrite')

desc_gmr.toPandas()

how much cities: 2250


,sub_region_1,n_cities,n_observations,min_date,max_date,nulls_census_fips_code,null_residential_percent_change_from_baseline,workplaces_percent_change_from_baseline,transit_stations_percent_change_from_baseline,parks_percent_change_from_baseline,retail_and_recreation_percent_change_from_baseline
0,RIO DE JANEIRO,85,36388,2020-02-15,2021-05-28,36388,12921,895.0,17468,14593,14025
1,RIO GRANDE DO SUL,175,69343,2020-02-15,2021-05-28,69343,34396,2503.0,47980,37427,33934
2,RONDONIA,23,8720,2020-02-15,2021-05-28,8720,4971,205.0,6917,6593,5568
3,MATO GROSSO,55,21283,2020-02-15,2021-05-28,21283,13063,384.0,15115,16370,14029
4,ESPIRITO SANTO,59,22202,2020-02-15,2021-05-28,22202,14158,622.0,16770,13865,13729
5,AMAZONAS,15,5612,2020-02-15,2021-05-28,5612,4253,141.0,5014,4105,3832
6,SERGIPE,28,10646,2020-02-15,2021-05-28,10646,7307,532.0,9211,7636,8537
7,RIO GRANDE DO NORTE,37,14938,2020-02-15,2021-05-28,14938,9921,739.0,10581,8384,10324
8,SANTA CATARINA,147,56699,2020-02-15,2021-05-28,56699,30248,2859.0,40431,36354,31412
9,PARAIBA,49,17593,2020-02-15,2021-05-28,17593,12976,1317.0,14891,13000,13167


In [13]:
gmr.write.csv('gs://ai-covid19-datalake/standard/google-mobility/pp_google-mobility_report-28-05-2021.csv', header=True, mode='overwrite')

<hr />

### municipality ibge data

<hr />

In [21]:
# read data from gcs
ibge = spark.read.csv('gs://ai-covid19-datalake/raw/ibge-data/ibge-municipality-to-code-28-05-2021.csv', inferSchema=True, header=True)

In [22]:
# show some lines
ibge.limit(5).toPandas()

,UF,Nome_UF,Mesorregião Geográfica,Nome_Mesorregião,Microrregião Geográfica,Nome_Microrregião,Município,mun_code,mun_name
0,11,Rondônia,2,Leste Rondoniense,6,Cacoal,15,1100015,Alta Floresta D'Oeste
1,11,Rondônia,2,Leste Rondoniense,6,Cacoal,379,1100379,Alto Alegre dos Parecis
2,11,Rondônia,2,Leste Rondoniense,3,Ariquemes,403,1100403,Alto Paraíso
3,11,Rondônia,2,Leste Rondoniense,5,Alvorada D'Oeste,346,1100346,Alvorada D'Oeste
4,11,Rondônia,2,Leste Rondoniense,3,Ariquemes,23,1100023,Ariquemes


In [23]:
# schema
ibge.printSchema()

root
 |-- UF: integer (nullable = true)
 |-- Nome_UF: string (nullable = true)
 |-- Mesorregião Geográfica: integer (nullable = true)
 |-- Nome_Mesorregião: string (nullable = true)
 |-- Microrregião Geográfica: integer (nullable = true)
 |-- Nome_Microrregião: string (nullable = true)
 |-- Município: integer (nullable = true)
 |-- mun_code: integer (nullable = true)
 |-- mun_name: string (nullable = true)



In [17]:
ibge = ibge.select(['UF', 'mun_code', 'Nome_UF', 'mun_name']).withColumnRenamed('Nome_UF', 'sub_region_1').withColumnRenamed('mun_name', 'sub_region_2')

In [18]:
# dropping NA in state and municipality
ibge = ibge.dropna(subset=['sub_region_1', 'sub_region_2'])

# taking accents and non-ascii chars
ibge = ibge.withColumn('sub_region_1', udf_remove_accents(F.col('sub_region_1')))
ibge = ibge.withColumn('sub_region_1', udf_normalize_ascii(F.col('sub_region_1')))

ibge = ibge.withColumn('sub_region_2', udf_remove_accents(F.col('sub_region_2')))
ibge = ibge.withColumn('sub_region_2', udf_normalize_ascii(F.col('sub_region_2')))


# turning columns into upper case
ibge = ibge.withColumn('sub_region_1', F.upper(F.col('sub_region_1')))
ibge = ibge.withColumn('sub_region_2', F.upper(F.col('sub_region_2')))

In [19]:
# show some lines
ibge.limit(5).toPandas()

,UF,mun_code,sub_region_1,sub_region_2
0,11,1100015,RONDONIA,ALTA FLORESTA D OESTE
1,11,1100379,RONDONIA,ALTO ALEGRE DOS PARECIS
2,11,1100403,RONDONIA,ALTO PARAISO
3,11,1100346,RONDONIA,ALVORADA D OESTE
4,11,1100023,RONDONIA,ARIQUEMES


In [20]:
ibge.write.csv('gs://ai-covid19-datalake/standard/ibge-data/pp_ibge-municipality-to-code-28-05-2021.csv', header=True, mode='overwrite')

<hr />

### meteorological data

<hr />

In [21]:
# getting the list of inmet files

client = storage.Client()
BUCKET_NAME = 'ai-covid19-datalake'
bucket = client.get_bucket(BUCKET_NAME)

blobs = bucket.list_blobs()
files = []
for blob in blobs:
    uri = 'raw/inmet-data/inmet-meteorological-data-01012019-a-28052021/'
    if blob.name[0:-38] == uri:
        files.append('gs://ai-covid19-datalake/'+blob.name)

In [29]:
inmet_schema = StructType([
    StructField('measurement_date', DateType()),
    StructField('total_daily_precipitation_mm', FloatType()),
    StructField('daily_atmospheric_pression_mb', FloatType()),
    StructField('daily_avg_dew_point_temp_c', FloatType()),
    StructField('max_daily_temp_maxima_diaria_c', FloatType()),
    StructField('daily_avg_temp_c', FloatType()),
    StructField('daily_min_temp_c', FloatType()),
    StructField('daily_avg_relative_air_humidity_percent', FloatType()),
    StructField('daily_min_relative_air_humidity_percent', FloatType()),
    StructField('max_gust_wind_ms', FloatType()),
    StructField('avg_wind_velocity_ms', FloatType()),
    StructField('empty', StringType())
    
])
    
inmet_header = ['measurement_date', 'total_daily_precipitation_mm', 'daily_atmospheric_pression_mb', 'daily_avg_dew_point_temp_c',
          'max_daily_temp_maxima_diaria_c', 'daily_avg_temp_c', 'daily_min_temp_c', 'daily_avg_relative_air_humidity_percent',
          'daily_min_relative_air_humidity_percent', 'max_gust_wind_ms', 'avg_wind_velocity_ms']

def replace_comma_and_convert(line):
    positions = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    date_positions = [0]
    
    for dp in date_positions:
        # converting string do proper date type, e.g.: '2019-01-02'
        if (line[dp] != 'null') and (line[dp] != ''):
            date_time_obj = datetime.datetime.strptime(line[dp], '%Y-%m-%d')
            line[dp] = date_time_obj.date()
        else:
            line[dp] = None

    
    for p in positions:
        if (line[p] != 'null') and (line[p] != ''):
            line[p] = float(line[p].replace(',', '.'))
        else:
            line[p] = None
    return line

In [30]:
dfs = []
for f in files:
    # reading file from list
    inmet_data = sc.textFile(f).map(lambda x: x.split(';'))
    
    # taking out blank lines (serious matter)
    inmet_data = inmet_data.filter(lambda x: x != [''])
    
    # getting the station ids 
    station_name = inmet_data.filter(lambda x: "Nome:" in x[0]).collect()[0][0].split(' ')[1]
    station_code = inmet_data.filter(lambda x: "Estacao:" in x[0]).collect()[0][0].split(' ')[2]
    
    # dropping the first 10 unnecessary top lines on file
    inmet_data = inmet_data.zipWithIndex().filter(lambda tup: tup[1] > 10).map(lambda x:x[0])
    
    # converting string to float values
    inmet_data = inmet_data.map(replace_comma_and_convert)
    
    # converting rdd to a dataframe with schema 
    df = spark.createDataFrame(inmet_data, schema=inmet_schema)
    
    # creating station and municipality variables
    df = df.withColumn('station_id', F.lit(station_code)).withColumn('municipality_name', F.lit(station_name))
    
    # appending new dataframe to be joined
    dfs.append(df)
    
inmet_df = reduce(DataFrame.unionAll, dfs)

In [31]:
# built dataset
inmet_df.limit(5).toPandas()

,measurement_date,total_daily_precipitation_mm,daily_atmospheric_pression_mb,daily_avg_dew_point_temp_c,max_daily_temp_maxima_diaria_c,daily_avg_temp_c,daily_min_temp_c,daily_avg_relative_air_humidity_percent,daily_min_relative_air_humidity_percent,max_gust_wind_ms,avg_wind_velocity_ms,empty,station_id,municipality_name
0,2019-01-02,0.0,888.766663,16.566668,29.200001,22.770832,16.900000,70.708336,38.0,8.3,2.062500,,A001,BRASILIA
1,2019-01-03,0.0,887.587524,14.141667,29.600000,24.429167,17.500000,55.291668,32.0,7.0,2.112500,,A001,BRASILIA
2,2019-01-04,0.0,887.554138,15.095833,28.900000,23.337500,18.700001,62.166668,37.0,7.9,1.820833,,A001,BRASILIA
3,2019-01-05,0.0,888.004150,16.650000,26.100000,20.762501,17.700001,78.000000,56.0,12.0,1.254167,,A001,BRASILIA
4,2019-01-06,1.0,887.633362,15.212500,28.200001,22.245832,18.000000,67.666664,36.0,7.5,2.187500,,A001,BRASILIA


In [33]:
# reading stations link file
stations_link = spark.read.csv('gs://ai-covid19-datalake/raw/inmet-data/inmet-estacoes-19-02-2018.csv', header=True).select(['station_id', 'sub_region_1'])

In [34]:
# list of stations and its states
stations_link.limit(5).toPandas()

,station_id,sub_region_1
0,A001,DISTRITO FEDERAL
1,A002,GOIAS
2,A003,GOIAS
3,A004,GOIAS
4,A008,DISTRITO FEDERAL


In [35]:
# bringing the state and municipality information
inmet_df = inmet_df.join(stations_link, 'station_id', 'left')

In [36]:
# now with the new state column added
inmet_df.limit(5).toPandas()

,station_id,measurement_date,total_daily_precipitation_mm,daily_atmospheric_pression_mb,daily_avg_dew_point_temp_c,max_daily_temp_maxima_diaria_c,daily_avg_temp_c,daily_min_temp_c,daily_avg_relative_air_humidity_percent,daily_min_relative_air_humidity_percent,max_gust_wind_ms,avg_wind_velocity_ms,empty,municipality_name,sub_region_1
0,A001,2019-01-02,0.0,888.766663,16.566668,29.200001,22.770832,16.900000,70.708336,38.0,8.3,2.062500,,BRASILIA,DISTRITO FEDERAL
1,A001,2019-01-03,0.0,887.587524,14.141667,29.600000,24.429167,17.500000,55.291668,32.0,7.0,2.112500,,BRASILIA,DISTRITO FEDERAL
2,A001,2019-01-04,0.0,887.554138,15.095833,28.900000,23.337500,18.700001,62.166668,37.0,7.9,1.820833,,BRASILIA,DISTRITO FEDERAL
3,A001,2019-01-05,0.0,888.004150,16.650000,26.100000,20.762501,17.700001,78.000000,56.0,12.0,1.254167,,BRASILIA,DISTRITO FEDERAL
4,A001,2019-01-06,1.0,887.633362,15.212500,28.200001,22.245832,18.000000,67.666664,36.0,7.5,2.187500,,BRASILIA,DISTRITO FEDERAL


In [ ]:
inmet_df.write.csv('gs://ai-covid19-datalake/standard/inmet-data/pp_inmet_meteorological_data-01012019-a-28052021', header=True, mode='overwrite')